In [1]:
import tensorflow as tf
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [2]:
!pip install transformers
#AKA huggingface library

     |████████████████████████████████| 645kB 4.3MB/s 
     |████████████████████████████████| 3.8MB 23.7MB/s 
     |████████████████████████████████| 890kB 54.1MB/s 
     |████████████████████████████████| 1.1MB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a522862f2c0c4c9e59054ec18134f73ffd1a7769e9f36b760127cee69147b3c5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install pytorch-lamb

     |████████████████████████████████| 204kB 3.7MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import string
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from pytorch_lamb import Lamb 
import tensorflow.keras.backend as K

TRAIN_FILE =  'drive/My Drive/DATA_MINING/DATA/train.csv'
TEST_FILE = 'drive/My Drive/DATA_MINING/DATA/test.csv'

train_data = pd.read_csv(TRAIN_FILE)        
test_data = pd.read_csv(TEST_FILE)

In [6]:
train_data.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [7]:
train_data.shape

(320552, 8)

**Load tokenizer**

In [8]:
# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True) #use bert-base-uncased for english version

# **If we use Bert english version else skip to the chinese part.**



In [0]:
def clean_data(data,col):
    clean_data = []
    for i in tqdm(range(data.shape[0])):
        clean_data.append(data.iloc[i, col].translate(str.maketrans("","",string.punctuation)).lower())
    data[str('CleanContent'+str(col-4))] = clean_data

clean_data(train_data,5)
clean_data(train_data,6)

In [0]:
CLS_token = '[CLS]'
SEP_token = '[SEP]'

def make_input(data):
  temp = []
  for i in tqdm(range(data.shape[0])):
    temp.append(str(CLS_token +data['CleanContent1'][i]+SEP_token+data['CleanContent2'][i]+SEP_token))
  data['INPUTS'] = temp
  
make_input(train_data)

In [0]:
tempo = []
for j in range(train_data.shape[0]):
  tempo.append(train_data["INPUTS"][j].split())

train_data["SPLITTED"] = tempo

In [0]:
max_len = max([len(k) for k in train_data['SPLITTED'].values])
min_len = min([len(k) for k in train_data['SPLITTED'].values])
mean_len = np.mean([len(k) for k in train_data['SPLITTED'].values])
perc_50 = np.percentile([len(k) for k in train_data['SPLITTED'].values],50)
print("max = {}, min = {}, mean = {}, 50% = {}".format(max_len, min_len, mean_len, perc_50))

#  **For chinese version : Preprocessing**


In [9]:
print(train_data['title2_zh'][48417])
train_data.fillna('undefined',inplace = True)
print(train_data['title2_zh'][48417])


nan
undefined


#  **Labels encoding**

In [0]:
labels = ['unrelated', 'agreed', 'disagreed']

temp=[]
def num_labels(data):
  for l in data['label']:
    temp.append(labels.index(l))
  data['NUM_LABEL'] = temp
num_labels(train_data)

**Search for the mean length of all the sequences (compare it to the median to avoid influence of strong outliers)**

In [0]:
max_len = []
sentences1 = train_data.title1_zh.values
sentences2 = train_data.title2_zh.values

for i in range(len(sentences1)):

    input_ids1 = tokenizer.encode(sentences1[i], add_special_tokens=True)
    input_ids2 = tokenizer.encode(sentences2[i], add_special_tokens=True)
    max_len.append(len(input_ids1)+ len((input_ids2)))

print('Mean sentence length: ', np.mean(max_len))

Mean sentence length:  52.1997429434226


 **Tokenize inputs**

In [11]:
input_ids = []
attention_masks = []
token_type_ids = []

for k in tqdm(range(train_data.shape[0])):

    encoded_dict = tokenizer.encode_plus(
                        train_data["title1_zh"][k],  
                        train_data["title2_zh"][k],                    
                        add_special_tokens = True, 
                        max_length = 64,  #We want it lowest as possible but still high enough to be relevant.         
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',  #pytorch tensors  
                   )
    
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_data.NUM_LABEL.values)
token_type_ids = torch.cat(token_type_ids, dim=0)


100%|██████████| 320552/320552 [03:00<00:00, 1779.94it/s]


**Observe the result for one example**

In [12]:
print('Original: ', '[CLS]'+train_data["title1_zh"][0] , "[SEP]", train_data["title2_zh"][0])
print('Token IDs:', input_ids[0])
print (attention_masks[0])
print(token_type_ids[0])


Original:  [CLS] 2017养老保险又新增两项，农村老人人人可申领，你领到了吗 [SEP] 警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京
Token IDs: tensor([ 101, 8109, 1075, 5439,  924, 7372, 1348, 3173, 1872,  697, 7555, 8024,
        1093, 3333, 5439,  782,  782,  782, 1377, 4509, 7566, 8024,  872, 7566,
        1168,  749, 1408,  102, 6356, 3175, 6792, 6469,  100, 7881, 2338, 1920,
         833, 3680,  782, 7566,  126,  674,  100,  793, 3300, 5439,  782, 1780,
        2898, 6822,  776,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


# **Prepare the model**


 **Split the training dataset for validation purpose**

In [0]:
dataset = TensorDataset(input_ids, attention_masks, labels, token_type_ids)
batch_size = 128

#Split training data into 80/20 for validation purpose
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

**Import the model and load it on GPU**

In [14]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-chinese", #"bert-base-uncased" for the english version
    num_labels = 3, # The number of output labels--3 for our classification. 
    output_attentions = False, 
    output_hidden_states = False, 
)

#Run the model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

**Set hyper-parameters values for tensorboard (not working)**

In [0]:
# We won't use that section because it takes to much time with our setup but it would be interesting to be able to test various parameters here
from sklearn import metrics
from tensorboard.plugins.hparams import api as hp

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3, 3e-5, 1e-5]))
HP_NB_EPOCHS = hp.HParam('epochs', hp.Discrete([1,2,4]))

**Initialize the optimizer and define a scheduler for the learning rate**

In [15]:
optimizer = Lamb(model.parameters(),
                  lr = 1e-3,
                  eps = 1e-8,
                  adam = False)

# optimizer = AdamW(model.parameters(),
#                   lr = 5e-5, #learning-rate
#                   eps = 1e-8)

epochs = 1

num_steps = int(len(train_dataloader) * epochs)
print("Steps :", num_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(num_steps/100),
                                            num_training_steps = num_steps-(int(num_steps/100)))


Steps : 2004


**Set the metrics to use**

In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(labels, preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


# **Model Training**

In [21]:
import random

# This training code is based on Chris McCormick's script which is based on "Transformers" example.

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 200 batches.
        if step % 200 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_token_type_ids = batch[3].to(device)

        model.zero_grad()        

        # Perform a forward pass
        loss, logits = model(b_input_ids, 
                             token_type_ids=b_token_type_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()


    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================


    print("")
    print("Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_precision = 0
    total_eval_recall = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_token_type_ids = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=b_token_type_ids, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            

        total_eval_loss += loss.item()


        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        acc = flat_accuracy(label_ids, logits)
        y_pred = np.argmax(logits, axis=1).flatten()
        prec = precision_score(label_ids, y_pred, average='micro')
        recall = recall_score(label_ids, y_pred, average='micro')

        total_eval_accuracy += acc
        total_eval_precision += prec
        total_eval_recall += recall



    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_precision = total_eval_precision / len(validation_dataloader)
    avg_val_recall = total_eval_recall / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Precision: {0:.2f}".format(avg_val_precision))
    print("  Recall: {0:.2f}".format(avg_val_recall))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch   200  of  2,004.    Elapsed: 0:02:28.
  Batch   400  of  2,004.    Elapsed: 0:04:56.
  Batch   600  of  2,004.    Elapsed: 0:07:24.
  Batch   800  of  2,004.    Elapsed: 0:09:53.
  Batch 1,000  of  2,004.    Elapsed: 0:12:21.
  Batch 1,200  of  2,004.    Elapsed: 0:14:49.
  Batch 1,400  of  2,004.    Elapsed: 0:17:17.
  Batch 1,600  of  2,004.    Elapsed: 0:19:45.
  Batch 1,800  of  2,004.    Elapsed: 0:22:13.
  Batch 2,000  of  2,004.    Elapsed: 0:24:41.

  Average training loss: 0.17
  Training epcoh took: 0:24:44

Validation...
  Accuracy: 0.90
  Precision: 0.90
  Recall: 0.90
  Validation Loss: 0.23
  Validation took: 0:01:50
Total training took 0:26:34 (h:mm:ss)


# **Testing the model**

In [22]:
 test_data.shape

(80126, 7)

In [0]:
test_data.fillna('undefined', inplace=True)
# Create sentence lists
title1 = test_data.title1_zh.values
title2 = test_data.title2_zh.values

**Tokenize test inputs**

In [24]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
token_type_ids = []


for k in tqdm(range(test_data.shape[0])):
    encoded_dict = tokenizer.encode_plus(
                        title1[k], 
                        title2[k],                     
                        add_special_tokens = True, 
                        max_length = 64,          
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
    
  
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, token_type_ids)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

100%|██████████| 80126/80126 [00:39<00:00, 2039.18it/s]


**Run the model with test set**

In [25]:
# Prediction on test set
predictions = []

# Put model in evaluation mode
model.eval()

for batch in tqdm(prediction_dataloader):
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_token_type_ids = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=b_token_type_ids, 
                      attention_mask=b_input_mask)

  logits = outputs[0]
  logits = logits.detach().cpu().numpy()
  predictions.append(logits)



100%|██████████| 2504/2504 [02:27<00:00, 17.00it/s]


# **Prepare data for submission (Kaggle)**

In [0]:
pred=[]

for i in range(len(predictions)):
  pred.append(np.argmax(predictions[i], axis=1).flatten())
  
tabl_predictions = []
categories = ["unrelated","agreed","disagreed"]
for i in range(len(pred)):
  for p in pred[i]:
    tabl_predictions.append(categories[p]) 

In [27]:
submission = pd.DataFrame()
submission['Id'] = test_data.id.values
submission['Category'] = tabl_predictions
submission.to_csv('sample_submission_chinese.csv',sep=',',index=False)
submission.head(50)

,Id,Category
0,321187,unrelated
1,321190,unrelated
2,321189,unrelated
3,321193,unrelated
4,321191,unrelated
5,321194,unrelated
6,321192,unrelated
7,321197,unrelated
8,321195,unrelated
9,321199,agreed
